# RQ4 - Prepare Historical Coocurrence - Initial slice

Uses:

- resources/historical_join.xlsx (HISTORICAL_FILE_JOIN from prepare/1.historical.ipynb)

Generates:

- resources/historical_rulesv1.xlsx


In [1]:
import sys  
sys.path.insert(1, '..')

In [2]:
!pip install mlxtend 

In [3]:
import pandas as pd
import numpy as np
import os
from util import HISTORICAL_FILE_JOIN, RESOURCE_DIR 
#pip install mlxtend
from mlxtend.frequent_patterns import apriori, association_rules

In [4]:
df = pd.read_excel(HISTORICAL_FILE_JOIN, keep_default_na=False)
df

,owner,name,domain,sha1,part_commit,date_commit,isLast,SapHana,Realm,SAP SQL Anywhere,...,Firebird,Cassandra,Couchbase,Etcd,PostGIS,Redis,Join_Ignite-Sql,Join_Ignite-NoSql,Join_Virtuoso-Sql,Join_Virtuoso-NoSql
0,Activiti,Activiti,Enterprise Resource Planning,7262dc3913045001cbc658a8474e9d295980c345,1,2010-07-14 06:36:18+00:00,False,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,Activiti,Activiti,Enterprise Resource Planning,38b284602be9dcb6b4253cfa1c2f9420c002b56a,2,2010-08-03 08:31:19+00:00,False,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,Activiti,Activiti,Enterprise Resource Planning,fbcdc18a66e7f7960a7ed966f87b1527c5c7551a,3,2010-09-02 09:45:24+00:00,False,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,Activiti,Activiti,Enterprise Resource Planning,8d05dda18ce4c5db6f30cf01d7e0c2059b1a5388,4,2010-09-16 15:33:33+00:00,False,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,Activiti,Activiti,Enterprise Resource Planning,a789ba2b30383777b4d0d81bbc62a90a24dbb032,5,2010-09-23 13:38:28+00:00,False,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
31111,liferay,liferay-portal,Enterprise Resource Planning,fcd4cd6526c80613eca69927effe931bb6a91444,8209,2024-10-07 12:05:23+00:00,False,0,0,0,...,0,0,0,0,1,1,0,0,0,0
31112,liferay,liferay-portal,Enterprise Resource Planning,cc0ab8103195e543056639c8c2883fcc187c33a2,8210,2024-10-07 17:58:58+00:00,False,0,0,0,...,0,0,0,0,1,1,0,0,0,0
31113,liferay,liferay-portal,Enterprise Resource Planning,f75082029372ab28694650bd3fe9fae4d548351e,8211,2024-10-11 04:36:52+00:00,False,0,0,0,...,0,0,0,0,1,1,0,0,0,0
31114,liferay,liferay-portal,Enterprise Resource Planning,a80a679a86d2867f985bafa510d9d12f1bf051a7,8212,2024-10-11 16:53:22+00:00,False,0,0,0,...,0,0,0,0,1,1,0,0,0,0


Grouping the lines to identify which projects have DB clues

In [5]:
df2 = df.groupby(["owner", "name"]).sum()
df2.drop(["part_commit", "isLast"], axis=1, inplace=True)
df2.drop(df2[(df2 == 0).all(axis=1)].index, inplace=True)
df2

,,SapHana,Realm,SAP SQL Anywhere,FileMaker,ArangoDB,Informix,HyperSQL,CouchDB,Riak KV,Neo4j,...,Firebird,Cassandra,Couchbase,Etcd,PostGIS,Redis,Join_Ignite-Sql,Join_Ignite-NoSql,Join_Virtuoso-Sql,Join_Virtuoso-NoSql
owner,name,,,,,,,,,,,,,,,,,,,,,
Activiti,Activiti,0,0,0,0,0,0,13,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Alluxio,alluxio,0,0,0,0,0,0,0,0,0,0,...,0,0,0,4,0,0,0,0,0,0
AnySoftKeyboard,AnySoftKeyboard,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Athou,commafeed,0,0,0,0,0,0,10,0,0,0,...,0,0,0,0,0,18,0,0,0,0
Atmosphere,atmosphere,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,26,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
wildfly,wildfly,0,0,0,0,0,0,203,0,0,0,...,0,0,0,0,0,0,0,0,0,0
xuxueli,xxl-job,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
yacy,yacy_search_server,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


New dataframe filtering only lines that are in the first slice of the projects

In [6]:
df10 = df[(df['part_commit'] == 1)].set_index(["owner", "name"])
df10.drop(["domain", "sha1", "part_commit", "date_commit", "isLast"], axis=1, inplace=True)
df10

SapHana  Realm  SAP SQL Anywhere  FileMaker  \
owner    name                                                                
Activiti Activiti                    0      0                 0          0   
airbnb   lottie-android              0      0                 0          0   
alibaba  spring-cloud-alibaba        0      0                 0          0   
         nacos                       0      0                 0          0   
         fastjson                    0      0                 0          0   
...                                ...    ...               ...        ...   
yacy     yacy_search_server          0      0                 0          0   
zaproxy  zaproxy                     0      0                 0          0   
zendesk  maxwell                     0      0                 0          0   
zxing    zxing                       0      0                 0          0   
liferay  liferay-portal              0      0                 0          0   

                               ArangoDB  Informix  HyperSQL  CouchDB  Riak KV  \
owner    name                                                                   
Activiti Activiti                     0         0         1        0        0   
airbnb   lottie-android               0         0         0        0        0   
alibaba  spring-cloud-alibaba         0         0         0        0        0   
         nacos                        0         0         0        0        0   
         fastjson                     0         0         0        0        0   
...                                 ...       ...       ...      ...      ...   
yacy     yacy_search_server           0         0         0        0        0   
zaproxy  zaproxy                      0         0         1        0        0   
zendesk  maxwell                      0         0         0        0        0   
zxing    zxing                        0         0         0        0        0   
liferay  liferay-portal               0         0         1        0        0   

                               Neo4j  ...  Firebird  Cassandra  Couchbase  \
owner    name                         ...                                   
Activiti Activiti                  0  ...         0          0          0   
airbnb   lottie-android            0  ...         0          0          0   
alibaba  spring-cloud-alibaba      0  ...         0          0          0   
         nacos                     0  ...         0          0          0   
         fastjson                  0  ...         0          0          0   
...                              ...  ...       ...        ...        ...   
yacy     yacy_search_server        0  ...         0          0          0   
zaproxy  zaproxy                   0  ...         0          0          0   
zendesk  maxwell                   0  ...         0          0          0   
zxing    zxing                     0  ...         0          0          0   
liferay  liferay-portal            0  ...         0          0          0   

                               Etcd  PostGIS  Redis  Join_Ignite-Sql  \
owner    name                                                          
Activiti Activiti                 0        0      0                0   
airbnb   lottie-android           0        0      0                0   
alibaba  spring-cloud-alibaba     0        0      0                0   
         nacos                    0        0      0                0   
         fastjson                 0        0      0                0   
...                             ...      ...    ...              ...   
yacy     yacy_search_server       0        0      0                0   
zaproxy  zaproxy                  0        0      0                0   
zendesk  maxwell                  0        0      0                0   
zxing    zxing                    0        0      0                0   
liferay  liferay-portal           0        0      0                0   

                       

Removing projects that don't have DB clues in the first slice 

In [7]:
with_dbs = df10.loc[df2.index].copy().astype('bool')
with_dbs

,,SapHana,Realm,SAP SQL Anywhere,FileMaker,ArangoDB,Informix,HyperSQL,CouchDB,Riak KV,Neo4j,...,Firebird,Cassandra,Couchbase,Etcd,PostGIS,Redis,Join_Ignite-Sql,Join_Ignite-NoSql,Join_Virtuoso-Sql,Join_Virtuoso-NoSql
owner,name,,,,,,,,,,,,,,,,,,,,,
Activiti,Activiti,False,False,False,False,False,False,True,False,False,False,...,False,False,False,False,False,False,False,False,False,False
Alluxio,alluxio,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
AnySoftKeyboard,AnySoftKeyboard,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
Athou,commafeed,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
Atmosphere,atmosphere,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
wildfly,wildfly,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
xuxueli,xxl-job,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
yacy,yacy_search_server,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False


### ASSOCIATION RULES

In [8]:
frq_items = apriori(with_dbs, min_support = 0.021, use_colnames = True)

In [9]:
rules = association_rules(frq_items, metric ="lift", min_threshold = 1)
rules['antecedent_len'] = rules['antecedents'].apply(lambda x: len(x))
rules['consequente_len'] = rules['consequents'].apply(lambda x: len(x))

In [10]:
# Sorting the rules by the lift measure 
rules = rules.sort_values('lift', ascending=False)
rules

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,zhangs_metric,antecedent_len,consequente_len
3159,"(HyperSQL, Oracle, MS SQL Server)","(IBM DB2, PostgreSQL, H2, MySQL)",0.025751,0.021459,0.021459,0.833333,38.833333,0.020907,5.871245,1.000000,3,4
1329,"(HyperSQL, Oracle, MS SQL Server)","(IBM DB2, H2)",0.025751,0.021459,0.021459,0.833333,38.833333,0.020907,5.871245,1.000000,3,2
2294,"(HyperSQL, Oracle, MS SQL Server, MySQL)","(IBM DB2, H2)",0.025751,0.021459,0.021459,0.833333,38.833333,0.020907,5.871245,1.000000,4,2
3152,"(HyperSQL, PostgreSQL, MS SQL Server)","(Oracle, IBM DB2, H2, MySQL)",0.025751,0.021459,0.021459,0.833333,38.833333,0.020907,5.871245,1.000000,3,4
2622,"(IBM DB2, PostgreSQL, H2)","(HyperSQL, Oracle, MS SQL Server)",0.021459,0.025751,0.021459,1.000000,38.833333,0.020907,inf,0.995614,3,3
...,...,...,...,...,...,...,...,...,...,...,...,...
23,(MySQL),(H2),0.231760,0.115880,0.072961,0.314815,2.716735,0.046105,1.290338,0.822544,1,1
9,(MySQL),(HyperSQL),0.231760,0.137339,0.085837,0.370370,2.696759,0.054007,1.370109,0.818994,1,1
8,(HyperSQL),(MySQL),0.137339,0.231760,0.085837,0.625000,2.696759,0.054007,2.048641,0.729353,1,1
50,(SQLite),(MySQL),0.051502,0.231760,0.030043,0.583333,2.516975,0.018107,1.843777,0.635423,1,1


Filtering only the rules of size 2, that is, 1 antecedent and 1 consequent. 

In [11]:
rules_filter = rules[(rules['antecedent_len'] == 1) & (rules['consequente_len'] == 1)]
rules_filter

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,zhangs_metric,antecedent_len,consequente_len
56,(IBM DB2),(MS SQL Server),0.042918,0.038627,0.021459,0.500000,12.944444,0.019801,1.922747,0.964126,1,1
57,(MS SQL Server),(IBM DB2),0.038627,0.042918,0.021459,0.555556,12.944444,0.019801,2.153433,0.959821,1,1
60,(SQLite),(MS SQL Server),0.051502,0.038627,0.021459,0.416667,10.787037,0.019470,1.648069,0.956561,1,1
61,(MS SQL Server),(SQLite),0.038627,0.051502,0.021459,0.555556,10.787037,0.019470,2.134120,0.943750,1,1
36,(Oracle),(IBM DB2),0.085837,0.042918,0.038627,0.450000,10.485000,0.034943,1.740148,0.989567,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...
23,(MySQL),(H2),0.231760,0.115880,0.072961,0.314815,2.716735,0.046105,1.290338,0.822544,1,1
9,(MySQL),(HyperSQL),0.231760,0.137339,0.085837,0.370370,2.696759,0.054007,1.370109,0.818994,1,1
8,(HyperSQL),(MySQL),0.137339,0.231760,0.085837,0.625000,2.696759,0.054007,2.048641,0.729353,1,1
50,(SQLite),(MySQL),0.051502,0.231760,0.030043,0.583333,2.516975,0.018107,1.843777,0.635423,1,1


Saving Results

In [12]:
HISTORICAL_RULES_FILE = RESOURCE_DIR + os.sep + 'historical_rulesv1.xlsx'  #'seq_patterns/testes/historical_rules_commit1_197reg_021.xlsx'
rules_filter = rules_filter.to_excel(HISTORICAL_RULES_FILE, index=False)